In [3]:
import dask.distributed as dd
from dask.distributed import Client, LocalCluster, progress
from dask_jobqueue import PBSCluster
from distributed.utils import tmpfile
from dask.distributed import get_worker
import os

/jobfs/93318185.gadi-pbs/ipykernel_203954/1276645085.py:4: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [4]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the workshop directory
# which was created in your username directory under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/"))

In [5]:
os.environ['DASK_PYTHON'] = '/scratch/vp91/AAPP2023/dask-python3.9-venv/bin/python3'

setup_commands = ["module load python3/3.9.2", "source /scratch/vp91/AAPP2023/dask-python3.9-venv/bin/activate"]

extra = ['-q normal',
         '-P vp91', 
         '-l ncpus=48', 
         '-l mem=192GB']

cluster = PBSCluster(walltime="00:50:00", 
                     cores=48, 
                     memory="192GB",
                     #processes=48, 
                     shebang='#!/usr/bin/env bash',
                     job_extra_directives=extra, 
                     local_directory='$TMPDIR', 
                     job_directives_skip=["select"], 
                     interface="ib0",
                     job_script_prologue=setup_commands,
                     python=os.environ["DASK_PYTHON"])

/scratch/vp91/AAPP2023/dask-python3.9-venv/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44799 instead
  warnings.warn(


In [6]:
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -l walltime=00:50:00
#PBS -q normal
#PBS -P vp91
#PBS -l ncpus=48
#PBS -l mem=192GB
module load python3/3.9.2
source /scratch/vp91/AAPP2023/dask-python3.9-venv/bin/activate
/scratch/vp91/AAPP2023/dask-python3.9-venv/bin/python3 -m distributed.cli.dask_worker tcp://10.6.43.71:43253 --nthreads 6 --nworkers 8 --memory-limit 22.35GiB --name dummy-name --nanny --death-timeout 60 --local-directory $TMPDIR --interface ib0



In [7]:
cluster.scale(jobs=2)

In [11]:
cluster

PBSCluster(0a305753, 'tcp://10.6.43.71:43253', workers=16, threads=96, memory=357.60 GiB)

In [10]:
!qstat

Job id                 Name             User              Time Use S Queue
---------------------  ---------------- ----------------  -------- - -----
93318185.gadi-pbs      sys-dashboard-s* jj8451            00:15:28 R normal-exec     
93321850.gadi-pbs      dask-worker      jj8451            00:00:00 R normal-exec     
93321862.gadi-pbs      dask-worker      jj8451            00:00:00 R normal-exec     


In [12]:
client = Client(cluster)

In [13]:
client

<Client: 'tcp://10.6.43.71:43253' processes=16 threads=96, memory=357.60 GiB>

In [14]:
def slow_increment(x):
    return x+1

In [15]:
futures = client.submit(slow_increment, 5000)

In [16]:
futures

<Future: finished, type: int, key: slow_increment-5cb046c889b1af847eb5ad207f9a4e9e>

In [17]:
progress(futures)

VBox()

In [18]:
import dask
import dask.array as da
from dask_ml.datasets import make_classification


n, d = 100000, 100

X, y = make_classification(n_samples=n, n_features=d,
                           chunks=n // 10, flip_y=0.2)
X

/scratch/vp91/AAPP2023/dask-python3.9-venv/lib/python3.9/site-packages/dask/base.py:1437: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


dask.array<normal, shape=(100000, 100), dtype=float64, chunksize=(10000, 100), chunktype=numpy.ndarray>

In [19]:
from dask_ml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train

dask.array<concatenate, shape=(90000, 100), dtype=float64, chunksize=(9000, 100), chunktype=numpy.ndarray>

In [20]:
from sklearn.linear_model import SGDClassifier

est = SGDClassifier(loss='squared_error', penalty='l2', tol=1e-3)

In [21]:
from dask_ml.wrappers import Incremental

inc = Incremental(est, scoring='accuracy')

In [22]:
import dask
X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

In [23]:
import dask.array as da
classes = da.unique(y_train).compute()
classes

array([0, 1])

In [24]:
inc.fit(X_train, y_train, classes=classes)

Incremental(estimator=SGDClassifier(loss='squared_error'), scoring='accuracy')

In [25]:
import joblib
with joblib.parallel_backend('dask'):
    inc.fit(X_train, y_train, classes=classes)